In [ ]:
---
title: "Assignment 18"
author: "Ryan Goyette"
date: "November 28, 2018"
output: html_document
---
#1
##Read in data, correct target variable type, and change target name.
```{r}
hmeq=read.csv("C:\\Users\\student\\Documents\\Senior\\R\\hmeq.csv", header=TRUE)
hmeq$BAD=as.factor(hmeq$BAD)
names(hmeq)[1]="target"
```

#2
##Quickly cleaning data to remove missing values.
```{r}
hmeq=hmeq[complete.cases(hmeq),]
```

#3
##Split data into 70/30 training/testing.
```{r}
##Split data
library(caret)
splitIndex<-createDataPartition(hmeq$target, p=.70, list=FALSE, times=1)
train<-hmeq[splitIndex,]
test<-hmeq[-splitIndex, ]
```

#4
##Training model.
```{r}
##Model
library(ranger)
model=ranger(target~., data=train)
pred=predict(model, data=test)$predictions
cm=confusionMatrix(pred, test$target, positive="1")
cm
```
###Accuracy=.9468
###Balanced Accuracy=.69565

#5
##Report ratio of default:non-default clients.
```{r}
table(hmeq$target)
prop.table(table(hmeq$target))
```
###91.21% of the data is non-defaulting clients.

#6
##Undersample to balance and build a model.
```{r}
##Balancing using undersampling
train1=train[train$target=="1", ]
train0=train[train$target=="0", ]
n1=nrow(train1)
n0=nrow(train0)
train00=train0[sample(1:n0, n1), ]
train_under=rbind(train00,train1)
##Model on undersampled
model_under=ranger(target~., data=train_under)
pred_under=predict(model_under, data=test)$predictions
cm_under=confusionMatrix(pred_under, test$target, positive="1")
cm_under
```
###Balanced Accuracy=.81274

#7
##Oversample to balance and build a model.
```{r}
##Balancing using oversampling
train1=train[train$target=="1", ]
train0=train[train$target=="0", ]
n1=nrow(train1)
n0=nrow(train0)
train11=train1[sample(1:n1, n0, replace=TRUE), ]
train_over=rbind(train0,train11)
##Model on oversampled
model_over=ranger(target~., data=train_over)
pred_over=predict(model_over, data=test)$predictions
cm_over=confusionMatrix(pred_over, test$target, positive="1")
cm_over
```
###Balanced Accuracy=.67883

#8
##Function: Inputs an unbalanced dataset with target variable named target and outputs a balanced dataset through under- or oversampling.
```{r}
###y=0: undersample; y=1: oversample
Quick_bal<-function(x,y)
{
  if(y==0)
  {
    x1=x[x$target=="1", ]
    x0=x[x$target=="0", ]
    n1=nrow(x1)
    n0=nrow(x0)
    if(n0>n1)
    {
      x00=x0[sample(1:n0, n1), ]
      x_under=rbind(x00,x1)
      return(x_under)
    }
    if(n0<n1)
    {
      x11=x1[sample(1:n1, n0), ]
      x_under=rbind(x11,x0)
      return(x_under)
    }
  }
  if(y==1)
  {
    x1=x[x$target=="1", ]
    x0=x[x$target=="0", ]
    n1=nrow(x1)
    n0=nrow(x0)
    if(n0>n1)
    {
      x11=x1[sample(1:n1, n0, replace=TRUE), ]
      x_over=rbind(x0,x11)
      return(x_over)
    }
    if(n0<n1)
    {
      x00=x0[sample(1:n0, n1, replace=TRUE), ]
      x_over=rbind(x1,x00)
      return(x_over)
    }
  }
}
```

#9
##Function creates a model
```{r}
Quick_model_bal<-function(x)
{
  library(caret)
  splitIndex<-createDataPartition(x$target, p=.70, list=FALSE, times=1)
  train<-x[splitIndex,]
  test<-x[-splitIndex, ]
  
  x1_under=x[x$target=="1", ]
  x0_under=x[x$target=="0", ]
  n1_under=nrow(x1_under)
  n0_under=nrow(x0_under)
  if(n0_under>n1_under)
  {
    x00_under=x0_under[sample(1:n0_under, n1_under), ]
    x_under=rbind(x00_under,x1_under)
    table_under=table(x_under$target)
  }
  if(n0_under<n1_under)
  {
    x11_under=x1_under[sample(1:n1_under, n0_under), ]
    x_under=rbind(x11_under,x0_under)
    table_under=table(x_under$target)
  }
  x1_over=x[x$target=="1", ]
  x0_over=x[x$target=="0", ]
  n1_over=nrow(x1_over)
  n0_over=nrow(x0_over)
  if(n0_over>n1_over)
  {
    x11_over=x1_over[sample(1:n1_over, n0_over, replace=TRUE), ]
    x_over=rbind(x0_over,x11_over)
    table_over=table(x_over$target)
  }
  if(n0_over<n1_over)
  {
    x00_over=x0_over[sample(1:n0_over, n1_over, replace=TRUE), ]
    x_over=rbind(x1_over,x00_over)
    table_over=table(x_over$target)
  }
  
  library(ranger)
  model_under=ranger(target~., data=x_under)
  pred_under=predict(model_under, data=test)$predictions
  cm_under=confusionMatrix(pred_under, test$target, positive="1")
  
  model_over=ranger(target~., data=x_over)
  pred_over=predict(model_over, data=test)$predictions
  cm_over=confusionMatrix(pred_over, test$target, positive="1")
  
  balacc_under=cm_under$byClass[11]
  balacc_over=cm_over$byClass[11]
  #print(table_under)
  #print(table_over)
  print("Underbalanced")
  print(balacc_under)
  print("Overbalanced")
  print(balacc_over)
}

Quick_model_bal(functest)